<a href="https://colab.research.google.com/github/machadoah/fatec-trabalho-rag-llamaindex/blob/main/myirirnotebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [83]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.linear_model import LogisticRegression
import matplotlib.pyplot as plt
import seaborn as sns
# Carregar dataset baixado
df = pd.read_csv("iris.data", header=None)

df.head()

,0,1,2,3,4
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa
3,4.6,3.1,1.5,0.2,Iris-setosa
4,5.0,3.6,1.4,0.2,Iris-setosa


In [84]:
# Carregar dataset baixado
df = pd.read_csv("iris.data")

df.head()


,5.1,3.5,1.4,0.2,Iris-setosa
0,4.9,3.0,1.4,0.2,Iris-setosa
1,4.7,3.2,1.3,0.2,Iris-setosa
2,4.6,3.1,1.5,0.2,Iris-setosa
3,5.0,3.6,1.4,0.2,Iris-setosa
4,5.4,3.9,1.7,0.4,Iris-setosa


In [85]:
df.describe()

df.info()



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 149 entries, 0 to 148
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   5.1          149 non-null    float64
 1   3.5          149 non-null    float64
 2   1.4          149 non-null    float64
 3   0.2          149 non-null    float64
 4   Iris-setosa  149 non-null    object 
dtypes: float64(4), object(1)
memory usage: 5.9+ KB


## 2. Pré-processamento de Dados (Transformação da Coluna Alvo)

Vamos transformar a coluna alvo (coluna `4`) de categórica (strings) para numérica, o que é essencial para o treinamento da maioria dos modelos de machine learning. Usaremos `LabelEncoder` para isso.

In [86]:
from sklearn.preprocessing import LabelEncoder

# Criar uma cópia do DataFrame para não modificar o original diretamente para futuras análises, se necessário
df_processed = df.copy()

# Inicializar o LabelEncoder
le = LabelEncoder()

# Aplicar o LabelEncoder à coluna alvo (a última coluna, que é a de índice -1)
# Usamos .iloc[:, -1] para garantir que a última coluna seja selecionada, independentemente do seu nome
df_processed.iloc[:, -1] = le.fit_transform(df_processed.iloc[:, -1])

print("Valores únicos da coluna alvo ANTES da transformação:")
print(df.iloc[:, -1].unique())

print("\nValores únicos da coluna alvo APÓS a transformação:")
print(df_processed.iloc[:, -1].unique())

print("\nPrimeiras 5 linhas do DataFrame após a transformação da coluna alvo:")
display(df_processed.head())

print("\nÚltimas 5 linhas do DataFrame após a transformação da coluna alvo:")
display(df_processed.tail())

print("\nVerificando os tipos de dados após a transformação:")
df_processed.info()

Valores únicos da coluna alvo ANTES da transformação:
['Iris-setosa' 'Iris-versicolor' 'Iris-virginica']

Valores únicos da coluna alvo APÓS a transformação:
[0 1 2]

Primeiras 5 linhas do DataFrame após a transformação da coluna alvo:


,5.1,3.5,1.4,0.2,Iris-setosa
0,4.9,3.0,1.4,0.2,0
1,4.7,3.2,1.3,0.2,0
2,4.6,3.1,1.5,0.2,0
3,5.0,3.6,1.4,0.2,0
4,5.4,3.9,1.7,0.4,0



Últimas 5 linhas do DataFrame após a transformação da coluna alvo:


,5.1,3.5,1.4,0.2,Iris-setosa
144,6.7,3.0,5.2,2.3,2
145,6.3,2.5,5.0,1.9,2
146,6.5,3.0,5.2,2.0,2
147,6.2,3.4,5.4,2.3,2
148,5.9,3.0,5.1,1.8,2



Verificando os tipos de dados após a transformação:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 149 entries, 0 to 148
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   5.1          149 non-null    float64
 1   3.5          149 non-null    float64
 2   1.4          149 non-null    float64
 3   0.2          149 non-null    float64
 4   Iris-setosa  149 non-null    object 
dtypes: float64(4), object(1)
memory usage: 5.9+ KB


## 3. Divisão dos Dados em Features e Alvo, e em Conjuntos de Treinamento e Teste

Agora que a coluna alvo está numérica, precisamos separar as colunas que serão usadas como *features* (variáveis de entrada) da coluna que é a *alvo* (o que queremos prever). Depois, dividiremos esses dados em conjuntos de treinamento e teste para avaliar o desempenho do modelo de forma justa.

In [87]:
from sklearn.model_selection import train_test_split

# Separar as features (X) da variável alvo (y)
# Todas as colunas, exceto a última (que é a alvo), são as features
X = df_processed.iloc[:, :-1]
# A última coluna é a variável alvo
y = df_processed.iloc[:, -1]

print("Shape das Features (X):", X.shape)
print("Shape da Variável Alvo (y):", y.shape)

# Dividir os dados em conjuntos de treinamento e teste
# test_size=0.3 significa 30% dos dados para teste, 70% para treinamento
# random_state garante que a divisão seja a mesma cada vez que você executar o código
# stratify=y garante que a proporção das classes da variável alvo seja a mesma nos conjuntos de treinamento e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)

print("\nShape de X_train (conjunto de treinamento de features):", X_train.shape)
print("Shape de X_test (conjunto de teste de features):", X_test.shape)
print("Shape de y_train (conjunto de treinamento da alvo):", y_train.shape)
print("Shape de y_test (conjunto de teste da alvo):", y_test.shape)

print("\nPrimeiras 5 linhas de X_train:")
display(X_train.head())

Shape das Features (X): (149, 4)
Shape da Variável Alvo (y): (149,)

Shape de X_train (conjunto de treinamento de features): (104, 4)
Shape de X_test (conjunto de teste de features): (45, 4)
Shape de y_train (conjunto de treinamento da alvo): (104,)
Shape de y_test (conjunto de teste da alvo): (45,)

Primeiras 5 linhas de X_train:


,5.1,3.5,1.4,0.2
88,5.5,2.5,4.0,1.3
25,5.0,3.4,1.6,0.4
138,6.9,3.1,5.4,2.1
29,4.8,3.1,1.6,0.2
19,5.4,3.4,1.7,0.2


# Task
Create a machine learning pipeline that includes the following steps: dividing the data into training, validation, and test sets; scaling the features using `StandardScaler`; training a Logistic Regression model; evaluating the model's performance on validation and test sets; and demonstrating how to make predictions on new data points.

## Divisão do Conjunto de Treinamento e Validação

### Subtask:
Dividir o conjunto de treinamento existente (`X_train`, `y_train`) em novos conjuntos de treinamento (`X_train_final`, `y_train_final`) e validação (`X_val`, `y_val`), além de adicionar uma célula de texto para explicar a importância dessa divisão.


## Importância da Divisão em Treinamento, Validação e Teste

Em Machine Learning, é crucial dividir o conjunto de dados em pelo menos três partes para garantir que o modelo seja robusto e generalize bem para dados não vistos:

1.  **Conjunto de Treinamento (`X_train_final`, `y_train_final`):** É a porção maior dos dados utilizada para "ensinar" o modelo. O modelo aprende padrões e relações a partir desses dados.

2.  **Conjunto de Validação (`X_val`, `y_val`):** Este conjunto é usado durante a fase de desenvolvimento do modelo para ajustar hiperparâmetros e tomar decisões de design do modelo (por exemplo, escolher a arquitetura do modelo, determinar o número de épocas de treinamento, etc.). Ele ajuda a evitar o *overfitting* (onde o modelo memoriza os dados de treinamento, mas falha em generalizar para novos dados) sem "contaminar" o conjunto de teste.

3.  **Conjunto de Teste (`X_test`, `y_test`):** Este é um conjunto totalmente independente de dados, que o modelo nunca "viu" durante o treinamento ou validação. Ele é usado apenas uma vez, no final do desenvolvimento do modelo, para uma avaliação imparcial do desempenho final do modelo e de sua capacidade de generalização.

**Reasoning**:
Now that the importance of splitting the data has been explained, the next step is to perform the actual split of the existing training data (`X_train`, `y_train`) into new training (`X_train_final`, `y_train_final`) and validation (`X_val`, `y_val`) sets using `train_test_split` with the specified parameters, and then print the shapes of these new sets.



In [88]:
from sklearn.model_selection import train_test_split

# Dividir o conjunto de treinamento existente em treinamento final e validação
# test_size=0.25 significa que 25% de X_train (original) irá para X_val
X_train_final, X_val, y_train_final, y_val = train_test_split(X_train, y_train, test_size=0.25, random_state=42, stratify=y_train)

print("Shape de X_train_final (novo conjunto de treinamento de features):", X_train_final.shape)
print("Shape de y_train_final (novo conjunto de treinamento da alvo):", y_train_final.shape)
print("Shape de X_val (conjunto de validação de features):", X_val.shape)
print("Shape de y_val (conjunto de validação da alvo):", y_val.shape)

Shape de X_train_final (novo conjunto de treinamento de features): (78, 4)
Shape de y_train_final (novo conjunto de treinamento da alvo): (78,)
Shape de X_val (conjunto de validação de features): (26, 4)
Shape de y_val (conjunto de validação da alvo): (26,)


## Escalonamento das Features

### Subtask:
Adicionar uma célula de texto para introduzir a necessidade de escalonar as features e uma célula de código Python para aplicar o `StandardScaler` aos conjuntos `X_train_final`, `X_val` e `X_test`, garantindo que os dados estejam na mesma escala.


## Escalonamento das Features

### Importância do Escalonamento de Features

Em muitos algoritmos de Machine Learning, especialmente aqueles que dependem da distância entre os pontos de dados (como K-Nearest Neighbors, Support Vector Machines) ou da magnitude das features (como Redes Neurais e Regressão Logística com regularização), é crucial que todas as features estejam na mesma escala.

Se as features tiverem escalas muito diferentes, aquelas com valores maiores podem dominar as com valores menores, levando o modelo a dar mais peso a features que não são necessariamente mais importantes. O escalonamento garante que cada feature contribua igualmente para o desempenho do modelo. Existem várias técnicas de escalonamento, sendo as mais comuns a normalização (Min-Max Scaling) e a padronização (Standardization).

Neste caso, utilizaremos a **Padronização (StandardScaler)**, que transforma os dados para que tenham média zero e desvio padrão unitário. Isso é feito subtraindo a média de cada feature e dividindo pelo seu desvio padrão. É importante ajustar o `StandardScaler` apenas aos dados de treinamento para aprender a média e o desvio padrão e, em seguida, aplicar essa mesma transformação aos conjuntos de validação e teste para evitar vazamento de dados.

**Reasoning**:
Now that the importance of feature scaling has been explained, the next step is to implement the scaling using `StandardScaler` on the `X_train_final`, `X_val`, and `X_test` datasets as per the instructions, and then display the head of the scaled training data.



In [89]:
from sklearn.preprocessing import StandardScaler

# Instanciar o StandardScaler
scaler = StandardScaler()

# Ajustar o scaler APENAS aos dados de treinamento final (X_train_final)
scaler.fit(X_train_final)

# Transformar os conjuntos de treinamento, validação e teste
X_train_scaled = scaler.transform(X_train_final)
X_val_scaled = scaler.transform(X_val)
X_test_scaled = scaler.transform(X_test)

# Converter os arrays NumPy resultantes de volta para DataFrames para melhor visualização, mantendo os nomes das colunas
X_train_scaled = pd.DataFrame(X_train_scaled, columns=X_train_final.columns, index=X_train_final.index)
X_val_scaled = pd.DataFrame(X_val_scaled, columns=X_val.columns, index=X_val.index)
X_test_scaled = pd.DataFrame(X_test_scaled, columns=X_test.columns, index=X_test.index)

print("Primeiras 5 linhas de X_train_scaled (dados de treinamento escalonados):")
display(X_train_scaled.head())

print("\nVerificando as estatísticas descritivas de X_train_scaled para confirmar o escalonamento:")
display(X_train_scaled.describe())

Primeiras 5 linhas de X_train_scaled (dados de treinamento escalonados):


,5.1,3.5,1.4,0.2
130,2.420074,1.709502,1.465137,1.026671
78,-0.216654,-0.988801,-0.157850,-0.256668
73,0.622305,-0.314225,0.289870,0.128334
114,0.622305,0.360350,0.849521,1.411673
29,-1.295316,0.135492,-1.221187,-1.283339



Verificando as estatísticas descritivas de X_train_scaled para confirmar o escalonamento:


,5.1,3.5,1.4,0.2
count,7.800000e+01,7.800000e+01,7.800000e+01,7.800000e+01
mean,6.220096e-16,1.340808e-15,6.262797e-17,8.255505e-17
std,1.006473e+00,1.006473e+00,1.006473e+00,1.006473e+00
min,-1.894573e+00,-2.337953e+00,-1.501012e+00,-1.411673e+00
25%,-6.660967e-01,-5.390840e-01,-1.263161e+00,-1.251256e+00
50%,-9.680298e-02,-8.936687e-02,2.339054e-01,1.283339e-01
75%,5.923420e-01,3.603503e-01,7.375912e-01,8.983374e-01
max,2.420074e+00,2.608936e+00,1.744963e+00,1.668341e+00


# Task
## 5. Treinamento do Modelo de Regressão Logística

### Subtask:
Adicionar uma célula de texto para introduzir o treinamento do modelo de Regressão Logística e uma célula de código Python para reinicializar e treinar o modelo usando os dados de treinamento escalonados (`X_train_scaled`, `y_train_final`).

## Treinamento do Modelo

### Subtask:
Reinicializar e treinar o modelo de Regressão Logística usando os dados de treinamento escalonados (`X_train_scaled`, `y_train_final`).


**Reasoning**:
The subtask is to initialize and train a Logistic Regression model. This step will initialize the model with a specified random state and then train it using the preprocessed training data.



In [91]:
from sklearn.linear_model import LogisticRegression

# 1. Inicializar um novo modelo de Regressão Logística
#    Definir random_state=42 para reprodutibilidade
model = LogisticRegression(random_state=42)

# 2. Treinar o modelo usando os dados de treinamento escalonados e os rótulos
model.fit(X_train_scaled, y_train_final)

print("Modelo de Regressão Logística treinado com sucesso!")

ValueError: Unknown label type: unknown. Maybe you are trying to fit a classifier, which expects discrete classes on a regression target with continuous values.

**Reasoning**:
The error `ValueError: Unknown label type: unknown` indicates that the target variable `y_train_final` is not in a suitable numerical format for the Logistic Regression classifier. Although `LabelEncoder` was used, the DataFrame column or Series might have retained an `object` dtype. Explicitly casting `y_train_final` to an integer type will resolve this issue.



In [ ]:
from sklearn.linear_model import LogisticRegression

# 1. Inicializar um novo modelo de Regressão Logística
#    Definir random_state=42 para reprodutibilidade
model = LogisticRegression(random_state=42)

# Converter y_train_final para tipo inteiro para evitar o erro de "Unknown label type"
y_train_final_int = y_train_final.astype(int)

# 2. Treinar o modelo usando os dados de treinamento escalonados e os rótulos
model.fit(X_train_scaled, y_train_final_int)

print("Modelo de Regressão Logística treinado com sucesso!")

## Avaliação Final do Modelo

### Subtask:
Fazer previsões nos conjuntos de validação (`X_val_scaled`, `y_val`) e teste (`X_test_scaled`, `y_test`), calcular e exibir métricas de avaliação (acurácia, classification report) e visualizar as matrizes de confusão para ambos os conjuntos.


**Reasoning**:
I need to import the necessary libraries for model evaluation and visualization, specifically `accuracy_score`, `classification_report`, `confusion_matrix` from `sklearn.metrics`, and `matplotlib.pyplot` and `seaborn` for plotting confusion matrices. This setup will allow me to perform all the requested evaluation steps.



In [ ]:
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns

print("Libraries for model evaluation and visualization imported.")

**Reasoning**:
Now that the necessary libraries are imported, I will proceed to make predictions on the validation and test sets, convert the true labels to integers, and then calculate and display the accuracy scores, classification reports, and confusion matrices for both sets to evaluate the model's performance.



In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# 1. Fazer previsões nos conjuntos de validação e teste
y_pred_val = model.predict(X_val_scaled)
y_pred_test = model.predict(X_test_scaled)

# Converter y_val e y_test para tipo inteiro, se ainda não forem, para garantir compatibilidade com as métricas
y_val_int = y_val.astype(int)
y_test_int = y_test.astype(int)

# 2. Avaliação no Conjunto de Validação
print("\nAvaliação no Conjunto de Validação:")
print("Acurácia na Validação:", accuracy_score(y_val_int, y_pred_val))
print("\nClassification Report (Validação):\n", classification_report(y_val_int, y_pred_val))

# 3. Avaliação no Conjunto de Teste
print("\n----------------------------------------------------")
print("Avaliação no Conjunto de Teste:")
print("Acurácia no Teste:", accuracy_score(y_test_int, y_pred_test))
print("\nClassification Report (Teste):\n", classification_report(y_test_int, y_pred_test))

# 4. Visualizar Matriz de Confusão para o Conjunto de Validação
cm_val = confusion_matrix(y_val_int, y_pred_val)
plt.figure(figsize=(8, 6))
sns.heatmap(cm_val, annot=True, fmt="d", cmap="Blues", cbar=False,
            xticklabels=model.classes_, yticklabels=model.classes_)
plt.title("Matriz de Confusão - Conjunto de Validação")
plt.xlabel("Previsão")
plt.ylabel("Real")
plt.show()

# 5. Visualizar Matriz de Confusão para o Conjunto de Teste
cm_test = confusion_matrix(y_test_int, y_pred_test)
plt.figure(figsize=(8, 6))
sns.heatmap(cm_test, annot=True, fmt="d", cmap="Greens", cbar=False,
            xticklabels=model.classes_, yticklabels=model.classes_)
plt.title("Matriz de Confusão - Conjunto de Teste")
plt.xlabel("Previsão")
plt.ylabel("Real")
plt.show()


## Previsão com o Modelo Treinado

### Subtask:
Demonstrar como usar o modelo treinado para fazer previsões em novos pontos de dados, incluindo a escalonagem desses novos dados antes da previsão.


**Reasoning**:
To demonstrate making predictions on new data, I need to create an example new data point, scale it using the previously fitted scaler, and then use the trained model to predict its class.



In [ ]:
import numpy as np

# 1. Criar um novo array NumPy representando um novo ponto de dado
# Usando valores que se assemelham a uma das classes (por exemplo, Iris-setosa)
novos_dados = np.array([[5.1, 3.5, 1.4, 0.2]]) # Exemplo de dados para um Iris-setosa

print("Novos dados (original):\n", novos_dados)

# 2. Escale os novos dados usando o scaler previamente ajustado
novos_dados_escalados = scaler.transform(novos_dados)

print("\nNovos dados (escalados):\n", novos_dados_escalados)

# 3. Use o modelo treinado para fazer previsões nos novos dados escalados
previsoes_novos_dados = model.predict(novos_dados_escalados)

# 4. Imprima as previsões
print("\nPrevisão para os novos dados (código numérico da classe):", previsoes_novos_dados[0])

# Opcional: Para interpretar a previsão, podemos usar o LabelEncoder inverso
# Certifique-se de que o 'le' (LabelEncoder) está acessível e ajustado
# Se 'le' não estiver acessível, você precisaria salvá-lo ou ter as mapeamento de classes

# Considerando que 'le' foi definido e ajustado anteriormente no notebook:
if 'le' in locals() and hasattr(le, 'inverse_transform'):
    nome_classe_prevista = le.inverse_transform(previsoes_novos_dados)
    print("Nome da classe prevista:", nome_classe_prevista[0])
else:
    print("LabelEncoder não disponível para traduzir a previsão.")

**Reasoning**:
The previous execution produced `UserWarning` messages because the `StandardScaler` and `LogisticRegression` model were fitted with feature names (from DataFrames), but new data was passed as a NumPy array without feature names. To resolve this, the new data will be converted to a DataFrame with appropriate column names before scaling and prediction, ensuring consistency.



In [ ]:
import numpy as np
import pandas as pd # Import pandas to create DataFrame

# 1. Criar um novo array NumPy representando um novo ponto de dado
# Usando valores que se assemelham a uma das classes (por exemplo, Iris-setosa)
novos_dados_array = np.array([[5.1, 3.5, 1.4, 0.2]]) # Exemplo de dados para um Iris-setosa

print("Novos dados (original):
", novos_dados_array)

# Converter o array para um DataFrame com os nomes das colunas corretos
# (assumindo que X_train_final.columns contêm os nomes das colunas)
novos_dados = pd.DataFrame(novos_dados_array, columns=X_train_final.columns)

# 2. Escale os novos dados usando o scaler previamente ajustado
novos_dados_escalados_array = scaler.transform(novos_dados)

# Converter os dados escalados de volta para um DataFrame para consistência com o modelo
novos_dados_escalados = pd.DataFrame(novos_dados_escalados_array, columns=X_train_final.columns)

print("\nNovos dados (escalados):
", novos_dados_escalados.values)

# 3. Use o modelo treinado para fazer previsões nos novos dados escalados
previsoes_novos_dados = model.predict(novos_dados_escalados)

# 4. Imprima as previsões
print("\nPrevisão para os novos dados (código numérico da classe):", previsoes_novos_dados[0])

# Opcional: Para interpretar a previsão, podemos usar o LabelEncoder inverso
# Certifique-se de que o 'le' (LabelEncoder) está acessível e ajustado
# Se 'le' não estiver acessível, você precisaria salvá-lo ou ter as mapeamento de classes

# Considerando que 'le' foi definido e ajustado anteriormente no notebook:
if 'le' in locals() and hasattr(le, 'inverse_transform'):
    nome_classe_prevista = le.inverse_transform(previsoes_novos_dados)
    print("Nome da classe prevista:", nome_classe_prevista[0])
else:
    print("LabelEncoder não disponível para traduzir a previsão.")

**Reasoning**:
The previous code block caused a `SyntaxError` due to unclosed string literals in the `print` statements. The string literals were broken across lines without proper multi-line string syntax or proper escaping. I will fix this by ensuring the string literals are correctly terminated on the same line and using `\n` for newlines within the printed output.



In [ ]:
import numpy as np
import pandas as pd # Import pandas to create DataFrame

print("--- Demonstrando Previsão com Novo Ponto de Dados ---")

# 1. Criar um novo ponto de dado para ser classificado
# Este é um exemplo de novas medidas de uma flor Iris que o modelo nunca viu.
# Os valores devem corresponder às 4 features usadas no treinamento.
novos_dados_array = np.array([[5.1, 3.5, 1.4, 0.2]]) # Exemplo: Características de uma Iris-setosa

print("\n1. Novos dados (original, antes do escalonamento):\n", novos_dados_array)

# Converter o array NumPy em um DataFrame Pandas.
# Isso é importante porque o 'scaler' e o 'model' foram ajustados com DataFrames que possuem nomes de colunas.
# Manter a consistência ajuda a evitar avisos (UserWarning) e garante que as features estejam na ordem correta.
novos_dados = pd.DataFrame(novos_dados_array, columns=X_train_final.columns)

# 2. Escalonar os novos dados usando o scaler previamente ajustado.
# É CRUCIAL usar o MESMO scaler (treinado apenas nos dados de treino) para transformar os novos dados.
# Isso garante que os novos dados estejam na mesma escala que o modelo 'espera' para fazer previsões precisas.
novos_dados_escalados_array = scaler.transform(novos_dados)

# O resultado do scaler.transform é um array NumPy. Para manter a consistência, convertemos de volta para um DataFrame.
novos_dados_escalados = pd.DataFrame(novos_dados_escalados_array, columns=X_train_final.columns)

print("\n2. Novos dados (escalados para corresponder ao treinamento do modelo):\n", novos_dados_escalados.values)

# 3. Usar o modelo treinado para fazer previsões nos novos dados escalados.
# O modelo 'model' (LogisticRegression) agora classifica o ponto de dados escalado.
previsoes_novos_dados = model.predict(novos_dados_escalados)

print("\n3. Previsão do modelo (código numérico da classe):", previsoes_novos_dados[0])

# 4. Traduzir a previsão numérica de volta para o nome original da classe (se o LabelEncoder estiver disponível).
# O LabelEncoder 'le' foi usado anteriormente para converter nomes em números. Agora, usamos seu inverso.
# Isso torna a previsão compreensível em termos das espécies de Iris originais.
if 'le' in locals() and hasattr(le, 'inverse_transform'):
    nome_classe_prevista = le.inverse_transform(previsoes_novos_dados)
    print("4. Nome da classe prevista (interpretado pelo LabelEncoder):", nome_classe_prevista[0])
else:
    print("LabelEncoder não disponível para traduzir a previsão. A previsão numérica é:", previsoes_novos_dados[0])
